In [1]:
#Install necessary libraries
!pip install groq
!pip install python-dotenv

In [2]:
import os
from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()

True

## OpenSource LLM Chatbot

In [3]:
from groq import Groq
import os

# Initialize the client with the API key
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

# Create chat completions with a system prompt
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant that always provides a single sentence as answer",
        },
        {
            "role": "user",
            "content": "How can I utilise an LLM?",
        }
    ],
    model="llama3-8b-8192", temperature = 0.5
)

# Print the response from the assistant
print(chat_completion.choices[0].message.content)

You can utilize a Large Language Model (LLM) by integrating it into your application or workflow to generate text, answer questions, or provide language translations.


# Asking LLM to provide output in a fixed format

In [4]:
import os
import json
from pydantic import BaseModel, Field, ValidationError
from typing import Optional, Union, Dict
from groq import Groq

# Define the structured output class with more flexible types
class Joke(BaseModel):
    """Facts to tell user."""
    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")
    awesome: Union[str, int, bool] = Field(description="Explanation of the joke")
    rating: Optional[Union[int, Dict[str, int]]] = Field(description="How funny the joke is, from 1 to 10 or a detailed rating dictionary")

def invoke_chat_completion(prompt: str) -> Optional[Joke]:
    # Initialize the client with the API key
    client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

    # Create chat completions with the user input
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant. Provide the response in JSON format with the following keys: setup, punchline, awesome, rating."
            },
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="llama3-8b-8192",
    )

    # Extract the response from the assistant
    response_content = chat_completion.choices[0].message.content
    #print("Raw response content:", response_content)

    try:
        # Parse the JSON response
        joke_data = json.loads(response_content)

        # Ensure the parsed data is a dictionary
        if not isinstance(joke_data, dict):
            raise ValueError("Parsed JSON is not a dictionary")

        # Convert boolean to string if necessary
        if isinstance(joke_data.get('awesome'), bool):
            joke_data['awesome'] = str(joke_data['awesome'])
        # Convert integer to string if necessary
        elif isinstance(joke_data.get('awesome'), int):
            joke_data['awesome'] = str(joke_data['awesome'])

        # Validate and create the Joke instance
        joke = Joke(
            setup=joke_data['setup'],
            punchline=joke_data['punchline'],
            awesome=joke_data['awesome'],
            rating=joke_data.get('rating')
        )

        return joke
    except (json.JSONDecodeError, ValidationError, ValueError) as e:
        print("Failed to parse response or validation error:", e)
        return None

# Example usage
prompt = "Why is Iceland green and Greenland white?"
joke = invoke_chat_completion(prompt)
if joke:
    print(joke)
else:
    print("Failed to retrieve a valid joke.")


setup='Why is Iceland green and Greenland white?' punchline='Because one is greener and the other is land!' awesome='True' rating=7


## Real time streaming

In [5]:
from groq import Groq
import os

# Initialize the client with the API key
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

# Function to invoke chat completion
def invoke_chat_completion(topic: str):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant. Write a 100 line poem about the given topic."
            },
            {
                "role": "user",
                "content": f"Write a poem about {topic}"
            }
        ],
        model="llama3-70b-8192",
        temperature=0, 
        stream= True
    )

    # Stream the response
    for chunk in chat_completion:
        print(chunk.choices[0].delta.content, end="", flush=True)

# Example usage
topic = "Sun moon and the goons"
invoke_chat_completion(topic)


In celestial harmony, they dance and play,
The Sun, the Moon, and the Goons, in a cosmic sway.
The Sun, a fiery ball of light and might,
Bringing warmth to the world, banishing the night.

The Moon, a glowing orb of gentle beam,
Illuminating dreams, and making hearts serene.
The Goons, a mischievous trio, full of glee,
Causing chaos and laughter, wherever they be.

With rays of sunshine, the Goons take flight,
Their antics and pranks, a wondrous, cosmic sight.
They chase the Moon, with a playful shout,
As she hides behind clouds, with a gentle pout.

The Sun, a watchful guardian, shines so bright,
Keeping the Goons in line, with a gentle might.
It guides them on their way, through the starry night,
As they frolic and play, with a celestial delight.

The Moon, a gentle mother, watches over all,
With a loving gaze, that soothes the heart's great fall.
She whispers secrets, to the Goons, so dear,
And they listen closely, with a reverent ear.

Together, the three, create a cosmic show,
A d